In [12]:
# Import Modules
import pandas as pd
import os
import json
import requests
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from pathlib import Path
import quandl
import numpy as np
import re

In [2]:
# Def som functions to re-use
def check_state_in_str(search_str):
  search_str_list = [x.strip() for x in search_str.split(';')]
  for x in search_str_list:
    if x in states:
      return x

def check_county_in_str(search_str):
  search_str_list = [x.strip() for x in search_str.split(';')]
  for x in search_str_list:
    if 'county' in x.lower():
      return x

def check_city_in_str(search_str):
  search_str_list = [x.strip() for x in search_str.split(';')]
  # if len is 1 then only zip code, if 5 then has all objects in str
  if len(search_str_list) == 1:
    return np.nan

  # else city will only occur at the end of the string
  # TODO: full proof way would be searching for metro name from FRED database
  if 'county' not in search_str_list[-1].lower():
    return search_str_list[-1]

def check_metro_in_str(search_str):
  search_str_list = [x.strip() for x in search_str.split(';')]
  # if len is 1 then only zip code, if 5 then has all objects in str
  if len(search_str_list) <= 3: # exploration: no metro in 3 objs or less
    return np.nan

  # else third positions should have metro
  if 'county' not in search_str_list[2].lower():
    return search_str_list[2]

In [3]:
# Load .env environment variables into the notebook
env_path = Path("quandl.env")
load_dotenv(env_path)

True

In [4]:
# Get the API key from the environment variable and store as Python variable
quandl_api_key = os.getenv("QUANDL_API_KEY")
type(quandl_api_key)

str

In [5]:
# configure quandl with the api key
quandl.ApiConfig.api_key = quandl_api_key

In [6]:
# get indicators
df_ind = quandl.get_table("ZILLOW/INDICATORS", paginate=True)
print('Num of indicators:', len(df_ind), '\n')

# view count of indicators by category
df_ind.groupby(['category'])['indicator_id'].count()

Num of indicators: 56 



category
Home values            10
Inventory and sales    44
Rentals                 2
Name: indicator_id, dtype: int64

In [7]:
# indicators for 'Home Values' category
df_ind.loc[df_ind['category'] == 'Home values']

,indicator_id,indicator,category
None,,,
0,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
1,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
2,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
3,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values
5,Z5BR,ZHVI 5+ Bedroom Time Series ($),Home values
6,Z4BR,ZHVI 4-Bedroom Time Series ($),Home values
7,Z3BR,ZHVI 3-Bedroom Time Series ($),Home values
8,Z2BR,ZHVI 2-Bedroom Time Series ($),Home values


In [16]:
# indicators for 'Rentals' category
df_ind.loc[df_ind['category'] == 'Rentals']

,indicator_id,indicator,category
None,,,
22,RSSA,"ZORI (Smoothed, Seasonally Adjusted): All Home...",Rentals
23,RSNA,ZORI (Smoothed): All Homes Plus Multifamily Ti...,Rentals


In [8]:
# get regions
df_regions = quandl.get_table("ZILLOW/REGIONS", paginate=True)
print('Num of regions:', len(df_regions))
print('Region types:', df_regions['region_type'].unique(), '\n')
df_regions.head()

Num of regions: 79252
Region types: ['zip' 'city' 'county' 'neigh' 'state' 'metro'] 



,region_id,region_type,region
None,,,
0,99999,zip,98847; WA; Wenatchee; Chelan County; Peshastin
1,99998,zip,98846; WA; Okanogan County; Pateros
2,99997,zip,98845; WA; Wenatchee; Douglas County; Palisades
3,99996,zip,98844; WA; Okanogan County; Oroville
4,99995,zip,98843; WA; Wenatchee; Douglas County; Orondo


In [13]:
##########################
# Normalize Regions Data #
##########################

# all states
states = ['IA', 'KS', 'UT', 'VA', 'NC', 'NE', 'SD', 'AL', 'ID', 'FM', 'DE', 'AK', 'CT', 'PR', 'NM', 'MS', 'PW', 'CO', 'NJ', 'FL', 'MN', 'VI', 'NV', 'AZ', 'WI', 'ND', 'PA', 'OK', 'KY', 'RI', 'NH', 'MO', 'ME', 'VT', 'GA', 'GU', 'AS', 'NY', 'CA', 'HI', 'IL', 'TN', 'MA', 'OH', 'MD', 'MI', 'WY', 'WA', 'OR', 'MH', 'SC', 'IN', 'LA', 'MP', 'DC', 'MT', 'AR', 'WV', 'TX']

# filter on region type
df_regions_zip = df_regions.loc[df_regions['region_type'] == 'zip']

# get all regions content
df_regions_zip['region_str_len'] = df_regions_zip.apply(lambda x: len(x['region'].split(';')), axis=1)
df_regions_zip['zip_code'] = df_regions_zip.apply(lambda x: re.search('(\d{5})', x['region']).group(), axis=1)
df_regions_zip['state'] = df_regions_zip.apply(lambda x: check_state_in_str(x['region']), axis=1)
df_regions_zip['county'] = df_regions_zip.apply(lambda x: check_county_in_str(x['region']), axis=1)
df_regions_zip['city'] = df_regions_zip.apply(lambda x: check_city_in_str(x['region']), axis=1)
df_regions_zip['metro'] = df_regions_zip.apply(lambda x: check_metro_in_str(x['region']), axis=1)
df_regions_zip.head()

C:\Users\saint\AppData\Local\Temp\ipykernel_24320\1785579450.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regions_zip['region_str_len'] = df_regions_zip.apply(lambda x: len(x['region'].split(';')), axis=1)
C:\Users\saint\AppData\Local\Temp\ipykernel_24320\1785579450.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regions_zip['zip_code'] = df_regions_zip.apply(lambda x: re.search('(\d{5})', x['region']).group(), axis=1)
C:\Users\saint\AppData\Local\Temp\ipykernel_24320\1785579450.py:14: Se

,region_id,region_type,region,region_str_len,zip_code,state,county,city,metro
None,,,,,,,,,
0,99999,zip,98847; WA; Wenatchee; Chelan County; Peshastin,5,98847,WA,Chelan County,Peshastin,Wenatchee
1,99998,zip,98846; WA; Okanogan County; Pateros,4,98846,WA,Okanogan County,Pateros,None
2,99997,zip,98845; WA; Wenatchee; Douglas County; Palisades,5,98845,WA,Douglas County,Palisades,Wenatchee
3,99996,zip,98844; WA; Okanogan County; Oroville,4,98844,WA,Okanogan County,Oroville,None
4,99995,zip,98843; WA; Wenatchee; Douglas County; Orondo,5,98843,WA,Douglas County,Orondo,Wenatchee


In [14]:
# view specific region
df_regions_zip.loc[
  (df_regions_zip['state'] == 'MD') & 
  (df_regions_zip['city'].str.contains('Takoma Park'))]

,region_id,region_type,region,region_str_len,zip_code,state,county,city,metro
None,,,,,,,,,
30253,66710,zip,20912; MD; Washington-Arlington-Alexandria; Mo...,5,20912,MD,Montgomery County,Takoma Park,Washington-Arlington-Alexandria


In [15]:
# median home price
# ZALL = ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)	
quandl.get_table('ZILLOW/DATA', indicator_id='ZALL', region_id='66710').head()

,indicator_id,region_id,date,value
None,,,,
0,ZALL,66710,2022-04-30,667198.0
1,ZALL,66710,2022-03-31,661292.0
2,ZALL,66710,2022-02-28,660443.0
3,ZALL,66710,2022-01-31,656388.0
4,ZALL,66710,2021-12-31,657264.0


In [17]:
# median rent price
# RSSA = ZORI (Smoothed, Seasonally Adjusted): All Homes Plus Multifamily Time Series ($)
quandl.get_table('ZILLOW/DATA', indicator_id='RSSA', region_id='66710').head()

,indicator_id,region_id,date,value
None,,,,


In [8]:

#url = "https://zillow56.p.rapidapi.com/search"

#querystring = {"location":"takoma park, md","isForRent":"false","isSingleFamily":"true","isMultiFamily":"false","isApartment":"false","isCondo":"false","isManufactured":"false","isLotLand":"false","doz":"36m"}

#headers = {
#	"X-RapidAPI-Host": "zillow56.p.rapidapi.com",
#	"X-RapidAPI-Key": rapidapi_api_key
#}

# Get and store response
#response = requests.request("GET", url, headers=headers, params=querystring).json()

# The response is of type dictionary
#type(response)

dict

In [9]:
# json.dumps is a convenient method for printing dict/json objects
#print(json.dumps(response, indent=3))

{
   "results": [
      {
         "bathrooms": 3.0,
         "bedrooms": 3.0,
         "city": "Takoma Park",
         "country": "USA",
         "currency": "USD",
         "daysOnZillow": -1,
         "homeStatus": "FOR_SALE",
         "homeStatusForHDP": "FOR_SALE",
         "homeType": "TOWNHOUSE",
         "isFeatured": false,
         "isNonOwnerOccupied": true,
         "isPreforeclosureAuction": false,
         "isPremierBuilder": false,
         "isUnmappable": false,
         "isZillowOwned": false,
         "latitude": 38.99631,
         "listing_sub_type": {
            "is_FSBA": true
         },
         "livingArea": 1620.0,
         "longitude": -76.994705,
         "lotAreaUnit": "sqft",
         "lotAreaValue": 1306.8,
         "price": 380000.0,
         "priceForHDP": 380000.0,
         "rentZestimate": 2669,
         "shouldHighlight": false,
         "state": "MD",
         "streetAddress": "36 Seek Ct",
         "taxAssessedValue": 270400.0,
         "zestimate"

In [34]:
#df = pd.json_normalize(response, record_path = ["results"])
#df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 39 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   bathrooms                           14 non-null     float64
 1   bedrooms                            14 non-null     float64
 2   city                                14 non-null     object 
 3   country                             14 non-null     object 
 4   currency                            14 non-null     object 
 5   daysOnZillow                        14 non-null     int64  
 6   homeStatus                          14 non-null     object 
 7   homeStatusForHDP                    14 non-null     object 
 8   homeType                            14 non-null     object 
 9   isFeatured                          14 non-null     bool   
 10  isNonOwnerOccupied                  14 non-null     bool   
 11  isPreforeclosureAuction             14 non-null